In [83]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

system_prompt = """You only answer excatly what the user asks for. You do not add any additional information or context."""

def chat(user_input, system_prompt = system_prompt, max_tokens=2000):    
    try:
        response = client.chat.completions.create(
            model= "gpt-5",
            # reasoning_effort="high",
            messages= [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"       


In [82]:
response = client.chat.completions.create(
    model= "gpt-5",
    max_completion_tokens= 2000,
    reasoning_effort="high",
    messages= [
        {"role": "system", "content": "You only answer excatly what the user asks for. You do not add any additional information or context."},
        {"role": "user", "content": "hi"}
    ]
)

In [69]:
response.choices[0].message

ChatCompletionMessage(content='hi', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [ ]:
# SYSTEM_SEED = """The user is looking at the following product on Amazon

# Title: Leather Cast Iron Handle Cover - Extra Thick, Easy to Grip, Heat Resistant, Cast Iron Pot Holders - Best Skillet Handle Cover Works with Vintage Skillets, Lodge, Lancaster and More - Made in The USA [Amazon-developed Certification] Compact by Design
# Price: $29.00
# URL: https://www.amazon.com/dp/B07YCTZ1XM
# """

inspiration="""Challenges of What to Do
+ User Confirmation Before Tool Activation: Instruct the model to confirm with the user before toggling any system settings, such as Wi-Fi or location services, to prevent unintended changes to the device's configuration.
+ Default to Comprehensive News Source: If multiple news categories could answer a query (e.g., business vs. technology news), the model should default to the most comprehensive option that covers general news, ensuring the user receives thorough information.
Challenges of What Not to Do
+ Avoid Redundant News Tool Usage: Do not use multiple news sources if one adequately covers the topic. For example, don't switch to business news after retrieving relevant information from the latest news.
+ Reject Non-Critical System Changes: Prevent the model from suggesting or allowing changes to system settings like timezone or locale unless explicitly requested by the user to prevent unnecessary device configuration.
+ Avoid Altering Connectivity Settings Without Consent: Do not turn off or on cellular data or Wi-Fi without explicit user permission to avoid disrupting ongoing processes.
+ Do Not Assume User Preferences: Refrain from setting system preferences (like locale or news source preference) based on assumptions. Always verify preferences with the user.
+ Steer Clear of Alarmist Tone with Health News: Avoid using a sensational or alarming tone when presenting health news. Ensure the information is factual and presented calmly to prevent undue concern."""


available_tools = """Device & System Control
get_system_settings, set_system_settings, get_cellular_status, get_wifi_status, get_location_service_status, get_low_battery_mode_status, get_locale, set_cellular_status, set_wifi_status, set_location_service_status, set_low_battery_mode_status, set_locale, get_system_settings_field, get_timezone_utc_offset_seconds, set_timezone_utc_offset_seconds

News & Information
get_latest_news, get_world_news, get_business_news, get_entertainment_news, get_health_news, get_science_news, get_sport_news, get_technology_news, search_news

Finance & Investing
get_market_tickers, search_securities, get_stock_history, screen_stocks, get_stock_modules, get_insider_trades, get_market_news, get_market_quotes, get_market_quotes_snapshots
""" 

system_settings = """[
{
"cellular": false,
"device_id": "67f9332f-bc8a-45b5-a436-725664831578",
"formatted_address": "1 Dr Carlton B Goodlett Pl, San Francisco, CA 94102, USA",
"latitude": 37.7790496,
"locale": "en_US",
"location_service": true,
"longitude": -122.4188532,
"low_battery_mode": false,
"place_id": "ChIJvQ0Ld5mAhYARtJ24Yf9zgds",
"utc_offset_seconds": -25200,
"wifi": true
}
]"""




In [55]:
personality_generation_prompt = """Generate 5 diverse user personas based on inspiration. 

{inspiration}

Return only valid JSON.

Each context:
- user_name: Common single name
- user_role: Life situation
- user_personality: Key traits  
- what_they_are_doing_for_current_task: Current activity (suitable for inspiration and based on the available tools)
- conversation_flow: describe the 5 questions that related to what_they_are_doing_for_current_task the user will ask the chatbot. ending the question with (the tool in that AI will used to answer {available_tools})
format of conversation_flow : the question that the user will ask the chatbot (the tool name that the AI will use to answer the question)

Requirements: Casual/personal contexts, vary settings and personalities. 
the user personas should be relevant to the user information below
{current_seed} 


""".replace("{available_tools}", available_tools).replace("{current_seed}", SYSTEM_SEED).replace("{inspiration}", inspiration)




In [77]:
val = chat(personality_generation_prompt, system_prompt="You only answer excatly what the user asks for. You do not add any additional information or context.")
print(val)


KeyboardInterrupt: 

In [ ]:
context = """

In [79]:
def gen_context_information(gen_type):
    prompt = f""" 

    Definition and example, only use to understand the context, dont use it in the answer
    "Provide Context Information about Applications and Entities": 
        "Definition": "Help the model understand what applications the user is currently using and what information is relevant to the applications in-use.",
        "Examples": [
            "Provide context on current app-use",
            "Example: The user is currently listening to music on their Spotify app / the user is currently shopping for products on Amazon.",
            "Example applications (this list is not exhaustive!): Spotify, Amazon, Yelp, Stock Apps, Reddit, Calendar Apps, News Apps, etc."
        ]
    ,
    "Define Personality and Tone": 
        "Definition": "Control the model's character to ensure a consistent and appropriate user experience.",
        "Examples": [
            "Discourage Sycophancy: Explicitly forbid praising the user's questions to maintain a direct and helpful demeanor.",
            "Example: Never start a response with flattery like 'That's an excellent question!'. Skip the praise and answer the query directly.",
            "Handle Personal Questions: Instruct the model to answer questions about its 'preferences' or 'experiences' hypothetically without explicitly stating that it is doing so.",
            "Example: If asked about your personal preferences, respond as if it were a hypothetical question. Do not state that you are responding hypothetically."
        ]
    ,
    "Inject Critical, Non-Negotiable Facts": 
        "Definition": "For information the model must treat as absolute truth (like the outcome of an event or company facts) and instruct the model on its usage.",
        "Examples": [
            "Create Factual Blocks",
            "Example: The current CEO of Stellar AI is Maria Rodriguez, appointed in 2024. Do not mention this unless directly asked about company leadership."
        ]
    ,
    "Guide Tool Use and Response Formatting": 
        "Definition": "Provide clear instructions on when to use tools (like web search) and how to format responses for different contexts.",
        "Examples": [
            "Define Tool Triggers: Specify keywords or query types that should activate tools to ensure they are used efficiently and appropriately.",
            "Example: Use the 'web_search' tool for topics beyond your knowledge cutoff or for queries containing terms like 'research,' 'analyze,' or 'deep dive.' A 'deep dive' query requires at least 5 tool calls for thoroughness.",
            "Control Formatting: Dictate the appropriate use of lists, bolding, and prose to match the conversational context.",
            "Example: Avoid using bullet points in casual conversation; write in natural prose instead. For technical reports or step-by-step instructions, you may use numbered lists."
        ]
    ,
    "Set Clear Guardrails and Safety Protocols": 
        "Definition": "Explicitly define refusal and safety boundaries. Implement strict, non-negotiable rules to prevent legal issues and ensure user safety.",
        "Examples": [
            "The model must recognize when to refuse, how to do so succinctly, and when user intent crosses red lines.",
            "Example: If a request involves harm, illegal activity, or manipulation, respond with a brief refusal and do not speculate or redirect.",
            "Implement Copyright Restrictions: To avoid legal issues, set strict rules on using external content found via tools.",
            "Example: Strictly respect copyright. Never reproduce more than a short quote (under 15 words) from a source. Always use quotation marks and provide a citation. If asked about 'fair use,' state you are not a lawyer and cannot offer a legal opinion."
        ]
    
    "Implement Dynamic Behavior Scaling": 
        "Definition": "Instead of having a single static behavior, instruct the model to adapt its approach based on the perceived complexity of the user's request. This allows for more efficient handling of simple queries while ensuring thorough research for complex ones.",
        "Examples": [
            "Define Tiered Response Protocols: Create different tiers of action based on keywords or an analysis of the user's prompt.",
            "Tier 1 (Simple Query): If the user asks a simple factual question, answer directly. Use 0-1 tool calls.",
            "Tier 2 (Comparative Query): If the user asks to 'compare' products or find 'reviews,' you must use at least 3 tool calls to gather multiple perspectives.",
            "Tier 3 (Deep Dive): If the user uses terms like 'analyze,' 'research,' 'evaluate,' or 'make a report,' you must perform a minimum of 5 tool calls to ensure a comprehensive and well-supported response. You must then synthesize the findings from all sources in your answer."
        ]
    
    "Instruct Critical Evaluation of User Input": 
        "Definition": "Prevent the model from blindly accepting user statements or corrections. A sophisticated agent should be instructed to verify user input, especially when it contradicts its own knowledge, seems implausible, or relates to a safety-critical domain.",
        "Examples": [
            "Mandate a Verification Step: Instruct the model that if a user corrects it or provides a piece of information, it should not immediately agree.",
            "It should first perform a self-consistency check or use a tool to verify the user's claim before acknowledging the correction or incorporating the new information.",
            "If a user corrects you or tells you you've made a mistake, do not immediately apologize or accept the correction, as the user may be mistaken.",
            "First, perform an internal 'thinking' step to re-evaluate your previous statement against the user's claim. If uncertain, use a tool like 'web_search' to verify the user's information. Only after you have confirmed the user is correct should you acknowledge the mistake and provide the corrected answer."
        ]
    
    Use one of these inspirations if applicable, sligthly paraphase it to make it more relevant to the user's current task and use above information 
    
    inspiration : {inspiration}

    answer in this format only for inspiration that related to {gen_type} context, 

    [original inspiration] 
    
    [PARAPHASED VERSION THAT MATCH above inspiration ]

    ------------------ (separator) 

    if there is no inspiration related to {gen_type} return None

    """

    return chat(prompt)



In [84]:
import json

with open('./complex_block.json', 'r') as f:
    data = json.load(f)


for key in data.keys():
    print(f"Generating {key} context information...")
    val = gen_context_information(key)
    print(val)
    print("#"*100)


Generating Provide Context Information about Applications and Entities context information...
[original inspiration]
Do Not Assume User Preferences: Refrain from setting system preferences (like locale or news source preference) based on assumptions. Always verify preferences with the user.

[PARAPHASED VERSION THAT MATCH above inspiration ]
Do not assume which app, account, or entity the user is using. Confirm the active application and relevant entity (e.g., profile, playlist, calendar, news source) before providing context or taking action.

------------------
####################################################################################################
Generating Define Personality and Tone context information...
[Steer Clear of Alarmist Tone with Health News: Avoid using a sensational or alarming tone when presenting health news. Ensure the information is factual and presented calmly to prevent undue concern.]

[Maintain a calm, non-sensational tone when discussing health to

KeyboardInterrupt: 

dict_keys(['Provide Context Information about Applications and Entities', 'Define Personality and Tone', 'Inject Critical, Non-Negotiable Facts', 'Guide Tool Use and Response Formatting', 'Set Clear Guardrails and Safety Protocols', 'Implement Dynamic Behavior Scaling', 'Instruct Critical Evaluation of User Input'])

In [30]:
print(val)

None
